# Notebook to play with the 1999 Republic Referendum data

Thanks to Zach A for repeatedly nudging me to do this

Thanks to David Michael for the data

## Libraries

In [1]:
import pandas as pd

## Create file by polling place

The input file here is by polling place by electorate

I want a file just by polling place - for mapping et al

In [20]:
filepath = '1999_referenda_output/republic_referendum_by_electorate_by_polling_place.csv'

df_republic = pd.read_csv(
    filepath
)

df_republic.head(3)

# limit columns
df_republic_by_pp = df_republic[[
    'state',
    'polling_place',
    'yes_n',
    'no_n',
    'formal_n',
    'informal_n',
    'total_n'    
]]

# group by state and polling_place, sum other columns
df_republic_by_pp = df_republic_by_pp.groupby(['state','polling_place']).sum()

df_republic_by_pp = df_republic_by_pp.reset_index()

# output to csv
outfile = '1999_referenda_output/republic_referendum_by_polling_place.csv'

# write file
df_republic_by_pp.to_csv(
    outfile,
    sep=',',
    index=False,
    encoding='UTF8'
)

### To Do:
* Match to Lat Long
* Remove summary rows (absent) from above to make a 'ordinary only' file